In [19]:
import pandas as pd
import numpy as np
from shapely.geometry import box
import geopandas as gpd

In [20]:
from pathlib import Path
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
BASE_DIR = Path(os.getenv("BASE_DIR"))
ELEVATION_STRINGS_PATH = os.getenv("ELEVATION_STRINGS_PATH")


In [6]:
MIN_STRINGS_PATH = os.getenv("MIN_STRINGS_PATH")
MAG_STRINGS_PATH =  os.getenv("MAG_STRINGS_PATH")


In [47]:
GEOLOGY_STRINGS_PATH = os.getenv("GEOLOGY_STRINGS_PATH")
GEOCHEMISTRY_STRINGS_PATH = os.getenv("GEOCHEMISTRY_STRINGS_PATH")

In [9]:
gpkg_path = BASE_DIR / os.getenv("MAG_STRINGS_PATH") / "geo_analyse.gpkg"

gdf_grid = gpd.read_file(
    gpkg_path,
    layer="grid_500m"
)

In [10]:
gdf_grid.head()

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,1vd_p90,dist_mag_line_m,mag_nearest_amp,mag_len_5km,mag_amp_max_5km,dist_grav_line_m,grav_nearest_amp,grav_len_5km,grav_amp_max_5km,geometry
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,-0.011257,7.266736e+06,4.092627,124753.390173,44.800618,8.111075,19.039854,190882.294656,39.498440,"POLYGON ((501931 7249146, 501931 7249646, 5014..."
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,-0.011989,7.267235e+06,4.092627,119626.656142,44.800618,304.773741,9.145121,188592.339952,39.498440,"POLYGON ((501931 7249646, 501931 7250146, 5014..."
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,-0.014655,7.267733e+06,4.092627,84948.625951,44.800618,671.610140,9.145121,185364.630506,39.465871,"POLYGON ((501931 7250146, 501931 7250646, 5014..."
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,-0.014378,7.268232e+06,4.092627,85881.737359,44.800618,982.790103,9.100928,176422.371620,35.294919,"POLYGON ((501931 7250646, 501931 7251146, 5014..."
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,-0.013610,7.268731e+06,4.092627,89694.261007,44.800618,1345.733665,9.100928,176422.371620,35.294919,"POLYGON ((501931 7251146, 501931 7251646, 5014..."


In [11]:
shp_path = BASE_DIR / MIN_STRINGS_PATH / "Strike" / "MINERALOCCURRENCES_shp" / "MINERALOCCURRENCES.shp"

gdf_mineral = gpd.read_file(shp_path)

In [14]:
gpd_grid = gdf_grid.copy()

In [15]:
gpd_grid.crs

<Projected CRS: EPSG:28352>
Name: GDA94 / MGA zone 52
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Australia - onshore and offshore between 126°E and 132°E.
- bounds: (125.99, -37.38, 132.0, -9.1)
Coordinate Operation:
- name: Map Grid of Australia zone 52
- method: Transverse Mercator
Datum: Geocentric Datum of Australia 1994
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [16]:
gdf_mineral = gdf_mineral.to_crs(gpd_grid.crs) 

In [17]:
buffer_m = 5000 
gdf_all = gdf_mineral.copy()
gdf_all = gdf_all.to_crs(gpd_grid.crs)
buf_polys = gdf_all.geometry.buffer(buffer_m)
any_buf_union = buf_polys.unary_union
gpd_grid["has_any_ozmin_5km"] = gpd_grid.geometry.intersects(any_buf_union).astype(int)

C:\Users\Phong\AppData\Local\Temp\ipykernel_80424\1660478645.py:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  any_buf_union = buf_polys.unary_union


In [18]:
gpd_grid

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,dist_mag_line_m,mag_nearest_amp,mag_len_5km,mag_amp_max_5km,dist_grav_line_m,grav_nearest_amp,grav_len_5km,grav_amp_max_5km,geometry,has_any_ozmin_5km
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,7.266736e+06,4.092627,124753.390173,44.800618,8.111075,19.039854,190882.294656,39.498440,"POLYGON ((501931 7249146, 501931 7249646, 5014...",0
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,7.267235e+06,4.092627,119626.656142,44.800618,304.773741,9.145121,188592.339952,39.498440,"POLYGON ((501931 7249646, 501931 7250146, 5014...",0
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,...,7.267733e+06,4.092627,84948.625951,44.800618,671.610140,9.145121,185364.630506,39.465871,"POLYGON ((501931 7250146, 501931 7250646, 5014...",0
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,...,7.268232e+06,4.092627,85881.737359,44.800618,982.790103,9.100928,176422.371620,35.294919,"POLYGON ((501931 7250646, 501931 7251146, 5014...",0
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,...,7.268731e+06,4.092627,89694.261007,44.800618,1345.733665,9.100928,176422.371620,35.294919,"POLYGON ((501931 7251146, 501931 7251646, 5014...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358531,0.0,5358531,-83.081830,0.137652,NaN,-38.844597,0.407874,NaN,-0.002623,0.000290,...,8.763723e+06,4.092627,30247.352570,2.484874,92194.648939,0.669765,0.000000,0.000000,"POLYGON ((1462431 8640646, 1462431 8641146, 14...",0
5358532,0.0,5358532,-83.039246,0.137805,NaN,-37.774031,0.347206,NaN,-0.001909,0.000117,...,8.764216e+06,4.092627,31957.857559,2.484874,92299.036318,0.669765,0.000000,0.000000,"POLYGON ((1462431 8641146, 1462431 8641646, 14...",0
5358533,0.0,5358533,-82.997389,0.162794,NaN,-36.845743,0.332489,NaN,-0.001805,0.000058,...,8.764709e+06,4.092627,31957.857559,2.484874,92406.011261,0.669765,0.000000,0.000000,"POLYGON ((1462431 8641646, 1462431 8642146, 14...",0
5358534,0.0,5358534,-83.021245,0.165225,NaN,-36.004838,0.345801,NaN,-0.002013,0.000089,...,8.765202e+06,4.092627,32778.972386,2.484874,92515.564795,0.669765,0.000000,0.000000,"POLYGON ((1462431 8642146, 1462431 8642646, 14...",0


In [21]:
df_dem_features = pd.read_parquet(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "features_topo_500m.parquet")


In [22]:
df_dem_features.head()

,dem_mean,dem_std,dem_p90,slope_mean,slope_std,slope_p90,aspect_sin_mean,aspect_cos_mean,roughness_mean,roughness_std,roughness_p90,curv_profile_mean,curv_profile_std,curv_profile_p90,x,y
0,0.174780,0.647749,0.000000,0.147078,0.548494,0.000000,-0.907985,-0.318529,0.190536,0.673838,0.000000,-0.000101,0.000422,0.000000,645435.774205,8.770492e+06
1,0.340703,0.851987,1.497947,0.492150,0.885583,2.083924,-0.877151,-0.390285,0.663777,1.099969,2.637327,-0.000242,0.000507,0.000000,645435.774205,8.770005e+06
2,5.498687,2.221002,8.131170,2.180369,1.292129,3.984081,-0.029759,0.209351,3.538564,1.646119,5.790140,0.000182,0.001213,0.001676,650306.553207,8.765134e+06
3,5.150033,1.763270,7.006257,1.740867,1.039413,3.048629,0.110908,0.158660,2.653270,1.348106,4.509090,0.000054,0.001024,0.001078,650793.631107,8.765134e+06
4,11.882605,5.267358,17.410387,1.907622,1.494803,4.028893,-0.527124,0.306119,2.969567,1.874631,5.831977,-0.000435,0.001023,0.000661,651280.709007,8.765134e+06


In [23]:
df_dem_features = gpd.GeoDataFrame(
    df_dem_features,
    geometry=gpd.points_from_xy(df_dem_features.x, df_dem_features.y),
    crs="EPSG:28352"
)

In [24]:
grid_dem = gpd.sjoin(
    gpd_grid,
    df_dem_features.drop(columns=['x', 'y']),
    how='left',
    predicate='contains'
)


In [25]:
# aggregate
dem_cols = [
    'dem_mean','dem_std','dem_p90',
    'slope_mean','slope_std','slope_p90',
    'aspect_sin_mean','aspect_cos_mean',
    'roughness_mean','roughness_std','roughness_p90',
    'curv_profile_mean','curv_profile_std','curv_profile_p90'
]

dem_agg = (
    grid_dem
    .groupby(grid_dem.index)[dem_cols]
    .mean()
)


In [26]:
gpd_grid[dem_cols] = dem_agg

In [27]:
gpd_grid.head(2)

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,slope_std,slope_p90,aspect_sin_mean,aspect_cos_mean,roughness_mean,roughness_std,roughness_p90,curv_profile_mean,curv_profile_std,curv_profile_p90
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,8.213200,26.75552,0.025196,0.277152,23.464565,11.646688,36.168823,-0.000840,0.004542,0.004276
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,6.033864,18.41120,-0.054660,0.540360,14.756773,8.317692,26.108307,-0.000593,0.002976,0.003267


In [28]:
# Check sanity
gpd_grid[dem_cols].isna().mean().sort_values(ascending=False)

dem_mean             0.099076
dem_std              0.099076
dem_p90              0.099076
slope_mean           0.099076
slope_std            0.099076
slope_p90            0.099076
aspect_sin_mean      0.099076
aspect_cos_mean      0.099076
roughness_mean       0.099076
roughness_std        0.099076
roughness_p90        0.099076
curv_profile_mean    0.099076
curv_profile_std     0.099076
curv_profile_p90     0.099076
dtype: float64

In [29]:
gpd_grid

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,slope_std,slope_p90,aspect_sin_mean,aspect_cos_mean,roughness_mean,roughness_std,roughness_p90,curv_profile_mean,curv_profile_std,curv_profile_p90
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,8.213200,26.755520,0.025196,0.277152,23.464565,11.646688,36.168823,-0.000840,0.004542,0.004276
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,6.033864,18.411200,-0.054660,0.540360,14.756773,8.317692,26.108307,-0.000593,0.002976,0.003267
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,...,1.662435,4.179825,-0.000368,0.519156,3.475145,2.133837,5.869995,-0.000170,0.001270,0.001269
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,...,1.420551,4.219540,-0.031954,0.177030,3.745028,1.776668,6.148132,0.000117,0.001509,0.001835
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,...,1.041703,3.250559,0.096753,0.229144,3.141291,1.239944,4.526886,-0.000072,0.001162,0.001475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358531,0.0,5358531,-83.081830,0.137652,NaN,-38.844597,0.407874,NaN,-0.002623,0.000290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5358532,0.0,5358532,-83.039246,0.137805,NaN,-37.774031,0.347206,NaN,-0.001909,0.000117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5358533,0.0,5358533,-82.997389,0.162794,NaN,-36.845743,0.332489,NaN,-0.001805,0.000058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5358534,0.0,5358534,-83.021245,0.165225,NaN,-36.004838,0.345801,NaN,-0.002013,0.000089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
gpd_grid.columns

Index(['label', 'grid_id', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean',
       'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std', '1vd_p90',
       'dist_mag_line_m', 'mag_nearest_amp', 'mag_len_5km', 'mag_amp_max_5km',
       'dist_grav_line_m', 'grav_nearest_amp', 'grav_len_5km',
       'grav_amp_max_5km', 'geometry', 'has_any_ozmin_5km', 'dem_mean',
       'dem_std', 'dem_p90', 'slope_mean', 'slope_std', 'slope_p90',
       'aspect_sin_mean', 'aspect_cos_mean', 'roughness_mean', 'roughness_std',
       'roughness_p90', 'curv_profile_mean', 'curv_profile_std',
       'curv_profile_p90'],
      dtype='object')

In [34]:
gpd_grid.geometry

0          POLYGON ((501931 7249146, 501931 7249646, 5014...
1          POLYGON ((501931 7249646, 501931 7250146, 5014...
2          POLYGON ((501931 7250146, 501931 7250646, 5014...
3          POLYGON ((501931 7250646, 501931 7251146, 5014...
4          POLYGON ((501931 7251146, 501931 7251646, 5014...
                                 ...                        
5358531    POLYGON ((1462431 8640646, 1462431 8641146, 14...
5358532    POLYGON ((1462431 8641146, 1462431 8641646, 14...
5358533    POLYGON ((1462431 8641646, 1462431 8642146, 14...
5358534    POLYGON ((1462431 8642146, 1462431 8642646, 14...
5358535    POLYGON ((1462431 8642646, 1462431 8643146, 14...
Name: geometry, Length: 5358536, dtype: geometry

In [30]:
out_gpkg  = BASE_DIR/MAG_STRINGS_PATH / "geo_analyse.gpkg"
geo_grid = gpd.read_file(out_gpkg, layer="grid_500m")

In [31]:
geo_grid

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,1vd_p90,dist_mag_line_m,mag_nearest_amp,mag_len_5km,mag_amp_max_5km,dist_grav_line_m,grav_nearest_amp,grav_len_5km,grav_amp_max_5km,geometry
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,-0.011257,7.266736e+06,4.092627,124753.390173,44.800618,8.111075,19.039854,190882.294656,39.498440,"POLYGON ((501931 7249146, 501931 7249646, 5014..."
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,-0.011989,7.267235e+06,4.092627,119626.656142,44.800618,304.773741,9.145121,188592.339952,39.498440,"POLYGON ((501931 7249646, 501931 7250146, 5014..."
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,-0.014655,7.267733e+06,4.092627,84948.625951,44.800618,671.610140,9.145121,185364.630506,39.465871,"POLYGON ((501931 7250146, 501931 7250646, 5014..."
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,-0.014378,7.268232e+06,4.092627,85881.737359,44.800618,982.790103,9.100928,176422.371620,35.294919,"POLYGON ((501931 7250646, 501931 7251146, 5014..."
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,-0.013610,7.268731e+06,4.092627,89694.261007,44.800618,1345.733665,9.100928,176422.371620,35.294919,"POLYGON ((501931 7251146, 501931 7251646, 5014..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358531,0.0,5358531,-83.081830,0.137652,NaN,-38.844597,0.407874,NaN,-0.002623,0.000290,NaN,8.763723e+06,4.092627,30247.352570,2.484874,92194.648939,0.669765,0.000000,0.000000,"POLYGON ((1462431 8640646, 1462431 8641146, 14..."
5358532,0.0,5358532,-83.039246,0.137805,NaN,-37.774031,0.347206,NaN,-0.001909,0.000117,NaN,8.764216e+06,4.092627,31957.857559,2.484874,92299.036318,0.669765,0.000000,0.000000,"POLYGON ((1462431 8641146, 1462431 8641646, 14..."
5358533,0.0,5358533,-82.997389,0.162794,NaN,-36.845743,0.332489,NaN,-0.001805,0.000058,NaN,8.764709e+06,4.092627,31957.857559,2.484874,92406.011261,0.669765,0.000000,0.000000,"POLYGON ((1462431 8641646, 1462431 8642146, 14..."
5358534,0.0,5358534,-83.021245,0.165225,NaN,-36.004838,0.345801,NaN,-0.002013,0.000089,NaN,8.765202e+06,4.092627,32778.972386,2.484874,92515.564795,0.669765,0.000000,0.000000,"POLYGON ((1462431 8642146, 1462431 8642646, 14..."


In [36]:
geo_grid.columns

Index(['label', 'grid_id', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean',
       'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std', '1vd_p90',
       'dist_mag_line_m', 'mag_nearest_amp', 'mag_len_5km', 'mag_amp_max_5km',
       'dist_grav_line_m', 'grav_nearest_amp', 'grav_len_5km',
       'grav_amp_max_5km', 'geometry'],
      dtype='object')

In [ ]:
exclude_cols = ["geometry", "label", "grid_id"]
cols_to_add = geo_grid.columns.difference(exclude_cols)

gpd_grid = pd.concat(
    [gpd_grid, geo_grid[cols_to_add]],
    axis=1
)

In [42]:
gpd_grid

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,grav_nearest_amp,mag_amp_max_5km,mag_len_5km,mag_nearest_amp,rtp_mean,rtp_p90,rtp_std,tmi_mean,tmi_p90,tmi_std
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,19.039854,44.800618,124753.390173,4.092627,-34.246340,-32.876038,1.099034,-341.819104,-339.439493,1.742753
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,9.145121,44.800618,119626.656142,4.092627,-32.579243,-31.281596,1.059147,-343.149000,-340.917496,1.733180
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,...,9.145121,44.800618,84948.625951,4.092627,-29.768549,-28.366481,1.097842,-344.533743,-342.339349,1.655959
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,...,9.100928,44.800618,85881.737359,4.092627,-26.194259,-24.623184,1.119967,-345.051959,-343.094437,1.430893
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,...,9.100928,44.800618,89694.261007,4.092627,-21.781616,-19.703737,1.419676,-345.027289,-343.312057,1.206863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358531,0.0,5358531,-83.081830,0.137652,NaN,-38.844597,0.407874,NaN,-0.002623,0.000290,...,0.669765,2.484874,30247.352570,4.092627,-38.844597,NaN,0.407874,-83.081830,NaN,0.137652
5358532,0.0,5358532,-83.039246,0.137805,NaN,-37.774031,0.347206,NaN,-0.001909,0.000117,...,0.669765,2.484874,31957.857559,4.092627,-37.774031,NaN,0.347206,-83.039246,NaN,0.137805
5358533,0.0,5358533,-82.997389,0.162794,NaN,-36.845743,0.332489,NaN,-0.001805,0.000058,...,0.669765,2.484874,31957.857559,4.092627,-36.845743,NaN,0.332489,-82.997389,NaN,0.162794
5358534,0.0,5358534,-83.021245,0.165225,NaN,-36.004838,0.345801,NaN,-0.002013,0.000089,...,0.669765,2.484874,32778.972386,4.092627,-36.004838,NaN,0.345801,-83.021245,NaN,0.165225


In [43]:
gpd_grid.columns

Index(['label', 'grid_id', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean',
       'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std', '1vd_p90',
       'dist_mag_line_m', 'mag_nearest_amp', 'mag_len_5km', 'mag_amp_max_5km',
       'dist_grav_line_m', 'grav_nearest_amp', 'grav_len_5km',
       'grav_amp_max_5km', 'geometry', 'has_any_ozmin_5km', 'dem_mean',
       'dem_std', 'dem_p90', 'slope_mean', 'slope_std', 'slope_p90',
       'aspect_sin_mean', 'aspect_cos_mean', 'roughness_mean', 'roughness_std',
       'roughness_p90', 'curv_profile_mean', 'curv_profile_std',
       'curv_profile_p90', '1vd_mean', '1vd_p90', '1vd_std',
       'dist_grav_line_m', 'dist_mag_line_m', 'grav_amp_max_5km',
       'grav_len_5km', 'grav_nearest_amp', 'mag_amp_max_5km', 'mag_len_5km',
       'mag_nearest_amp', 'rtp_mean', 'rtp_p90', 'rtp_std', 'tmi_mean',
       'tmi_p90', 'tmi_std'],
      dtype='object')

In [57]:
folder = Path(BASE_DIR / GEOCHEMISTRY_STRINGS_PATH / "processed")

files = {
    "rock": folder / "rock_cleaned.parquet",
    "soil": folder / "soil_cleaned.parquet",
    "stream_sediment": folder / "stream_sediment_cleaned.parquet",
}

In [58]:
def read_parquet_auto(path: Path):
    
    try:
        g = gpd.read_parquet(path)
        
        if "geometry" in g.columns:
            return g
        return pd.read_parquet(path)
    except Exception:
        # fallback pandas
        return pd.read_parquet(path)

dfs = {}
for name, path in files.items():
    df = read_parquet_auto(path)
    dfs[name] = df
    print(f"\n=== {name} ===")
    print("path:", path)
    print("type:", type(df))
    print("shape:", df.shape)
    print("columns (first 20):", list(df.columns[:20]))
    if isinstance(df, gpd.GeoDataFrame):
        print("crs:", df.crs)

rock_df = dfs["rock"]
soil_df = dfs["soil"]
stream_df = dfs["stream_sediment"]


=== rock ===
path: C:\Users\Phong\Desktop\GIS\Project 2\5_Geochemistry\processed\rock_cleaned.parquet
type: <class 'geopandas.geodataframe.GeoDataFrame'>
shape: (151749, 24)
columns (first 20): ['UNIQ_ID', 'ID', 'SAMPLEID', 'SAMPLEREF', 'SAM_TYPE', 'SAM_METH', 'MAP100K', 'MAP250K', 'LONGITUDE', 'LATITUDE', 'geometry', 'r_CU_PPM', 'r_AU_PPM', 'r_AG_PPM', 'r_AS_PPM', 'r_MO_PPM', 'r_PB_PPM', 'r_ZN_PPM', 'r_CO_PPM', 'r_NI_PPM']
crs: {"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "GeographicCRS", "name": "GDA94", "datum": {"type": "GeodeticReferenceFrame", "name": "Geocentric Datum of Australia 1994", "ellipsoid": {"name": "GRS 1980", "semi_major_axis": 6378137, "inverse_flattening": 298.257222101}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "scope": "Horizontal

In [59]:
rock_df.head()


,UNIQ_ID,ID,SAMPLEID,SAMPLEREF,SAM_TYPE,SAM_METH,MAP100K,MAP250K,LONGITUDE,LATITUDE,...,r_AS_PPM,r_MO_PPM,r_PB_PPM,r_ZN_PPM,r_CO_PPM,r_NI_PPM,r_SB_PPM,r_W_PPM,r_BI_PPM,r_SN_PPM
0,71,0AD1696_4787764,4787764,ML00CJE087,ROCK,WROCK,Haast Bluff,MOUNT LIEBIG,131.77648,-23.20574,...,NaN,0.4,26.0,38.0,5.0,5.0,NaN,0.4,NaN,NaN
1,72,0AD1696_4787779,4787779,ML00CJE134,ROCK,WROCK,Haast Bluff,MOUNT LIEBIG,131.59288,-23.31091,...,0.5,NaN,24.0,18.5,2.7,4.0,NaN,0.3,NaN,NaN
2,73,0AD1696_4791519,4791519,ML00IRS113,ROCK,WROCK,Haast Bluff,MOUNT LIEBIG,131.83262,-23.22533,...,2.0,0.7,21.0,89.0,8.5,6.0,NaN,2.8,0.4,7.0
3,74,0AD1696_4791534,4791534,ML00IRS130,ROCK,WROCK,Haast Bluff,MOUNT LIEBIG,131.59302,-23.25547,...,2.0,NaN,4.5,72.0,55.0,360.0,NaN,1.4,0.4,NaN
4,75,0AD1696_4791549,4791549,ML00IRS130A,ROCK,WROCK,Haast Bluff,MOUNT LIEBIG,131.59302,-23.25547,...,4.0,0.3,23.5,38.5,4.5,5.0,NaN,1.0,0.3,NaN


In [60]:
rock_feat_cols = [
    "r_CU_PPM", "r_AU_PPM", "r_AG_PPM", "r_AS_PPM",
    "r_MO_PPM", "r_PB_PPM", "r_ZN_PPM",
    "r_CO_PPM", "r_NI_PPM", "r_SB_PPM",
    "r_W_PPM", "r_BI_PPM", "r_SN_PPM"
]

rock_feat_cols = [c for c in rock_feat_cols if c in rock_df.columns]
print("Rock features used:", rock_feat_cols)

Rock features used: ['r_CU_PPM', 'r_AU_PPM', 'r_AG_PPM', 'r_AS_PPM', 'r_MO_PPM', 'r_PB_PPM', 'r_ZN_PPM', 'r_CO_PPM', 'r_NI_PPM', 'r_SB_PPM', 'r_W_PPM', 'r_BI_PPM', 'r_SN_PPM']


In [61]:
if rock_df.crs != gpd_grid.crs:
    rock_df = rock_df.to_crs(gpd_grid.crs)

In [62]:
rock_join = gpd.sjoin(
    rock_df[["geometry"] + rock_feat_cols],
    gpd_grid[["grid_id", "geometry"]],
    how="left",
    predicate="within"
)

In [63]:
agg_dict = {c: "mean" for c in rock_feat_cols}
agg_dict["geometry"] = "count"   # dùng geometry để đếm sample

rock_grid = (
    rock_join
    .groupby("grid_id")
    .agg(agg_dict)
    .rename(columns={"geometry": "r_rock_count"})
    .reset_index()
)

# rename feature columns → *_mean
rock_grid = rock_grid.rename(
    columns={c: f"{c}_mean" for c in rock_feat_cols}
)

In [64]:
gpd_grid_feat = gpd_grid.merge(
    rock_grid,
    on="grid_id",
    how="left"
)


In [65]:
gpd_grid_feat.describe()

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,r_MO_PPM_mean,r_PB_PPM_mean,r_ZN_PPM_mean,r_CO_PPM_mean,r_NI_PPM_mean,r_SB_PPM_mean,r_W_PPM_mean,r_BI_PPM_mean,r_SN_PPM_mean,r_rock_count
count,5.358536e+06,5.358536e+06,5.075891e+06,5.075891e+06,5.075891e+06,5.076491e+06,5.076491e+06,5.076491e+06,5.076491e+06,5.076491e+06,...,8449.000000,18410.000000,17864.000000,11399.000000,11575.000000,6163.000000,6765.000000,8890.000000,7188.000000,27262.000000
mean,1.425160e-02,2.679268e+06,-6.693826e+01,6.377348e+00,-5.859444e+01,1.146577e+01,7.612324e+00,2.145659e+01,3.460255e-05,2.581609e-02,...,21.383118,533.488320,241.553763,35.929294,57.853601,65.312239,175.537859,38.333072,79.497054,5.506786
std,1.081983e-01,1.546876e+06,1.694646e+02,1.730856e+01,1.752340e+02,1.988214e+02,2.083102e+01,2.080418e+02,1.128894e-01,8.891928e-02,...,958.622748,9043.084312,3696.559800,286.993007,1004.555389,2002.208818,2356.894476,423.189337,2424.888290,42.379767
min,0.000000e+00,0.000000e+00,-1.980966e+03,0.000000e+00,-1.617727e+03,-2.727466e+03,0.000000e+00,-2.167288e+03,-7.414126e+00,0.000000e+00,...,0.000000,0.000500,0.000500,0.000500,0.001100,0.000030,0.000000,0.000000,0.000500,1.000000
25%,0.000000e+00,1.339634e+06,-1.360887e+02,7.792242e-01,-1.307799e+02,-8.845759e+01,9.367311e-01,-8.284424e+01,-1.153668e-02,1.319484e-03,...,0.450000,6.874578,9.764423,2.350000,5.000000,0.250000,0.400000,0.100000,0.900000,1.000000
50%,0.000000e+00,2.679268e+06,-7.738126e+01,1.898976e+00,-7.341091e+01,-1.455147e+01,2.243029e+00,-8.859299e+00,-1.372288e-03,4.060122e-03,...,1.100000,17.142857,30.000000,8.000000,12.454545,0.750000,1.380000,0.470000,2.736667,2.000000
75%,0.000000e+00,4.018901e+06,-9.815431e+00,5.172586e+00,-3.426247e+00,7.853616e+01,6.187425e+00,8.631475e+01,7.214037e-03,1.620304e-02,...,3.360000,46.500000,87.000000,25.000000,37.444444,3.525897,5.500000,4.322500,7.200000,3.000000
max,1.000000e+00,5.358535e+06,4.641514e+03,1.192614e+03,5.360796e+03,4.848545e+03,1.550656e+03,5.794384e+03,1.063437e+01,8.804346e+00,...,84309.144786,500000.000000,351000.000000,15706.000000,106000.000000,122750.000000,69320.000000,20100.000000,198700.000000,3708.000000


In [66]:
soil_df.head()

,geometry,s_CU_PPM_median_pt,s_CU_PPM_log1p_median_pt,s_AS_PPM_median_pt,s_AU_PPB_median_pt,s_MO_PPM_median_pt,s_SB_PPM_median_pt,s_ZN_PPM_median_pt,s_PB_PPM_median_pt,s_CO_PPM_median_pt,...,s_AU_PPB_bdl_rate_pt,s_AS_PPM_bdl_rate_pt,s_MO_PPM_bdl_rate_pt,s_SB_PPM_bdl_rate_pt,s_ZN_PPM_bdl_rate_pt,s_PB_PPM_bdl_rate_pt,TITLE,COMPANY,MAP250K,MAP100K
0,POINT (129.13888 -25.13135),10.5,2.418141,1.1,0.65,<NA>,<NA>,87.5,13.0,8.05,...,0.0,0.0,0.0,0.0,0.0,0.0,EL5702,Independence Gold NL,PETERMANN RANGES,Pottoyu
1,POINT (129.13889 -25.13587),12.5,2.567899,1.15,0.6,<NA>,<NA>,69.5,10.5,10.35,...,0.0,0.0,0.0,0.0,0.0,0.0,EL5702,Independence Gold NL,PETERMANN RANGES,Pottoyu
2,POINT (129.13889 -25.14038),9.0,2.282174,0.95,1.7,<NA>,<NA>,64.0,12.5,11.75,...,0.0,0.0,0.0,0.0,0.0,0.0,EL5702,Independence Gold NL,PETERMANN RANGES,Pottoyu
3,POINT (129.1389 -25.1449),10.5,2.418141,1.25,0.8,<NA>,<NA>,71.0,13.0,14.2,...,0.0,0.0,0.0,0.0,0.0,0.0,EL5702,Independence Gold NL,PETERMANN RANGES,Pottoyu
4,POINT (129.1389 -25.14941),11.0,2.470821,1.1,0.7,<NA>,<NA>,77.0,14.0,18.1,...,0.0,0.0,0.0,0.0,0.0,0.0,EL5702,Independence Gold NL,PETERMANN RANGES,Pottoyu


In [67]:
soil_feat_cols = [
    "s_CU_PPM_median_pt",
    "s_CU_PPM_log1p_median_pt",
    "s_AS_PPM_median_pt",
    "s_AU_PPB_median_pt",
    "s_MO_PPM_median_pt",
    "s_SB_PPM_median_pt",
    "s_ZN_PPM_median_pt",
    "s_PB_PPM_median_pt",
    "s_CO_PPM_median_pt",
    "s_NI_PPM_median_pt",
    "s_FE_PCT_median_pt",
    "s_S_PCT_median_pt",
    "s_CU_PPM_bdl_rate_pt",
    "s_AS_PPM_bdl_rate_pt",
    "s_AU_PPB_bdl_rate_pt",
    "s_MO_PPM_bdl_rate_pt",
    "s_SB_PPM_bdl_rate_pt",
    "s_ZN_PPM_bdl_rate_pt",
    "s_PB_PPM_bdl_rate_pt",
]

soil_feat_cols = [c for c in soil_feat_cols if c in soil_df.columns]
print("Soil features used:", soil_feat_cols)


Soil features used: ['s_CU_PPM_median_pt', 's_CU_PPM_log1p_median_pt', 's_AS_PPM_median_pt', 's_AU_PPB_median_pt', 's_MO_PPM_median_pt', 's_SB_PPM_median_pt', 's_ZN_PPM_median_pt', 's_PB_PPM_median_pt', 's_CO_PPM_median_pt', 's_NI_PPM_median_pt', 's_FE_PCT_median_pt', 's_S_PCT_median_pt', 's_CU_PPM_bdl_rate_pt', 's_AS_PPM_bdl_rate_pt', 's_AU_PPB_bdl_rate_pt', 's_MO_PPM_bdl_rate_pt', 's_SB_PPM_bdl_rate_pt', 's_ZN_PPM_bdl_rate_pt', 's_PB_PPM_bdl_rate_pt']


In [68]:
if soil_df.crs != gpd_grid_feat.crs:
    soil_df = soil_df.to_crs(gpd_grid_feat.crs)

In [69]:
soil_join = gpd.sjoin(
    soil_df[["geometry"] + soil_feat_cols],
    gpd_grid_feat[["grid_id", "geometry"]],
    how="left",
    predicate="within"
)

In [70]:
agg_dict = {c: "mean" for c in soil_feat_cols}
agg_dict["geometry"] = "count"

soil_grid = (
    soil_join
    .groupby("grid_id")
    .agg(agg_dict)
    .rename(columns={"geometry": "s_soil_count"})
    .reset_index()
)

# rename feature columns → *_mean
soil_grid = soil_grid.rename(
    columns={c: f"{c}_mean" for c in soil_feat_cols}
)


In [71]:
gpd_grid_feat = gpd_grid_feat.merge(
    soil_grid,
    on="grid_id",
    how="left"
)

In [72]:
gpd_grid_feat.head()

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,s_FE_PCT_median_pt_mean,s_S_PCT_median_pt_mean,s_CU_PPM_bdl_rate_pt_mean,s_AS_PPM_bdl_rate_pt_mean,s_AU_PPB_bdl_rate_pt_mean,s_MO_PPM_bdl_rate_pt_mean,s_SB_PPM_bdl_rate_pt_mean,s_ZN_PPM_bdl_rate_pt_mean,s_PB_PPM_bdl_rate_pt_mean,s_soil_count
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
stream_df.head()

,geometry,ss_cu_ppm,ss_pb_ppm,ss_zn_ppm,ss_as_ppm,ss_mo_ppm,ss_co_ppm,ss_ni_ppm,ss_u_ppm,ss_au_ppb,ss_ag_ppm,ss_sb_ppm,ss_fe_pct,ss_mn_ppm,ss_cr_ppm,ss_has_any
0,POINT (1254083.274 7473836.407),1.301030,1.000000,1.477121,0.301030,-0.30103,0.778151,1.113943,0.30103,-3.0,-1.301030,0.397940,4.468347,2.447158,2.204120,1
1,POINT (699935.058 8424016.674),NaN,1.475671,NaN,0.602060,-1.00000,NaN,NaN,NaN,NaN,-0.387216,-0.397940,NaN,NaN,NaN,1
2,POINT (700658.557 8421767.904),NaN,1.698101,NaN,0.602060,-1.30103,NaN,NaN,NaN,NaN,-0.055517,-0.387216,NaN,NaN,NaN,1
3,POINT (693558.297 8427056.454),0.795185,0.974972,1.220108,1.113943,-0.30103,0.515874,0.532754,NaN,NaN,-0.420216,-0.267606,4.315970,2.089905,1.278754,1
4,POINT (1223072.957 7478143.546),NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.30103,NaN,NaN,NaN,NaN,NaN,NaN,1


In [74]:
stream_feat_cols = [
    "ss_cu_ppm",
    "ss_pb_ppm",
    "ss_zn_ppm",
    "ss_as_ppm",
    "ss_mo_ppm",
    "ss_co_ppm",
    "ss_ni_ppm",
    "ss_u_ppm",
    "ss_au_ppb",
    "ss_ag_ppm",
    "ss_sb_ppm",
    "ss_fe_pct",
    "ss_mn_ppm",
    "ss_cr_ppm",
    "ss_has_any"
]

stream_feat_cols = [c for c in stream_feat_cols if c in stream_df.columns]
print("Stream features used:", stream_feat_cols)


Stream features used: ['ss_cu_ppm', 'ss_pb_ppm', 'ss_zn_ppm', 'ss_as_ppm', 'ss_mo_ppm', 'ss_co_ppm', 'ss_ni_ppm', 'ss_u_ppm', 'ss_au_ppb', 'ss_ag_ppm', 'ss_sb_ppm', 'ss_fe_pct', 'ss_mn_ppm', 'ss_cr_ppm', 'ss_has_any']


In [75]:
if stream_df.crs != gpd_grid_feat.crs:
    stream_df = stream_df.to_crs(gpd_grid_feat.crs)

In [76]:
stream_join = gpd.sjoin(
    stream_df[["geometry"] + stream_feat_cols],
    gpd_grid_feat[["grid_id", "geometry"]],
    how="left",
    predicate="within"
)

In [77]:
agg_dict = {c: "mean" for c in stream_feat_cols}
agg_dict["geometry"] = "count"

stream_grid = (
    stream_join
    .groupby("grid_id")
    .agg(agg_dict)
    .rename(columns={"geometry": "ss_stream_count"})
    .reset_index()
)

# rename feature columns → *_mean
stream_grid = stream_grid.rename(
    columns={c: f"{c}_mean" for c in stream_feat_cols}
)


In [78]:
gpd_grid_feat = gpd_grid_feat.merge(
    stream_grid,
    on="grid_id",
    how="left"
)


In [79]:
gpd_grid_feat.describe()

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,ss_ni_ppm_mean,ss_u_ppm_mean,ss_au_ppb_mean,ss_ag_ppm_mean,ss_sb_ppm_mean,ss_fe_pct_mean,ss_mn_ppm_mean,ss_cr_ppm_mean,ss_has_any_mean,ss_stream_count
count,5.358536e+06,5.358536e+06,5.075891e+06,5.075891e+06,5.075891e+06,5.076491e+06,5.076491e+06,5.076491e+06,5.076491e+06,5.076491e+06,...,29060.000000,20036.000000,29605.000000,25511.000000,15616.000000,28164.000000,30459.000000,22775.000000,64672.000000,64672.000000
mean,1.425160e-02,2.679268e+06,-6.693826e+01,6.377348e+00,-5.859444e+01,1.146577e+01,7.612324e+00,2.145659e+01,3.460255e-05,2.581609e-02,...,0.977721,0.285861,-3.036821,-0.878958,-0.148692,4.131167,2.288854,1.321483,0.993256,2.434547
std,1.081983e-01,1.546876e+06,1.694646e+02,1.730856e+01,1.752340e+02,1.988214e+02,2.083102e+01,2.080418e+02,1.128894e-01,8.891928e-02,...,0.471733,0.530739,0.886843,0.739676,0.760573,0.879921,0.725156,0.782349,0.079528,4.614744
min,0.000000e+00,0.000000e+00,-1.980966e+03,0.000000e+00,-1.617727e+03,-2.727466e+03,0.000000e+00,-2.167288e+03,-7.414126e+00,0.000000e+00,...,-1.397940,-2.207608,-4.602060,-3.698970,-3.602060,0.079181,-0.397940,-3.000000,0.000000,1.000000
25%,0.000000e+00,1.339634e+06,-1.360887e+02,7.792242e-01,-1.307799e+02,-8.845759e+01,9.367311e-01,-8.284424e+01,-1.153668e-02,1.319484e-03,...,0.698970,0.007055,-3.552842,-1.301030,-0.602060,4.037061,1.929419,1.000000,1.000000,1.000000
50%,0.000000e+00,2.679268e+06,-7.738126e+01,1.898976e+00,-7.341091e+01,-1.455147e+01,2.243029e+00,-8.859299e+00,-1.372288e-03,4.060122e-03,...,1.018972,0.301030,-3.301030,-0.657577,0.000000,4.281033,2.361728,1.397940,1.000000,2.000000
75%,0.000000e+00,4.018901e+06,-9.815431e+00,5.172586e+00,-3.426247e+00,7.853616e+01,6.187425e+00,8.631475e+01,7.214037e-03,1.620304e-02,...,1.301030,0.602060,-2.602060,-0.301030,0.397940,4.521138,2.732394,1.724276,1.000000,3.000000
max,1.000000e+00,5.358535e+06,4.641514e+03,1.192614e+03,5.360796e+03,4.848545e+03,1.550656e+03,5.794384e+03,1.063437e+01,8.804346e+00,...,2.107210,2.176091,0.000000,0.602060,1.397940,5.600488,4.255273,2.885077,1.000000,423.000000


In [80]:
gpd_grid_feat.columns

Index(['label', 'grid_id', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean',
       'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std',
       ...
       'ss_ni_ppm_mean', 'ss_u_ppm_mean', 'ss_au_ppb_mean', 'ss_ag_ppm_mean',
       'ss_sb_ppm_mean', 'ss_fe_pct_mean', 'ss_mn_ppm_mean', 'ss_cr_ppm_mean',
       'ss_has_any_mean', 'ss_stream_count'],
      dtype='object', length=102)

In [81]:
# df = gpd_grid_feat.drop(columns=["geometry"]).copy()
df = gpd_grid_feat.copy()
# # 1) drop constant-ish columns
# const_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
# print("Constant cols:", const_cols)
# df = df.drop(columns=const_cols)

# 2) add has-data indicators
if "r_rock_count" in df.columns:
    df["has_rock"] = (df["r_rock_count"].fillna(0) > 0).astype(int)
if "s_soil_count" in df.columns:
    df["has_soil"] = (df["s_soil_count"].fillna(0) > 0).astype(int)
if "ss_stream_count" in df.columns:
    df["has_stream"] = (df["ss_stream_count"].fillna(0) > 0).astype(int)

print("Final shape for ML:", df.shape)

Final shape for ML: (5358536, 105)


In [82]:
df.columns

Index(['label', 'grid_id', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean',
       'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std',
       ...
       'ss_ag_ppm_mean', 'ss_sb_ppm_mean', 'ss_fe_pct_mean', 'ss_mn_ppm_mean',
       'ss_cr_ppm_mean', 'ss_has_any_mean', 'ss_stream_count', 'has_rock',
       'has_soil', 'has_stream'],
      dtype='object', length=105)

In [83]:
df.head()

,label,grid_id,tmi_mean,tmi_std,tmi_p90,rtp_mean,rtp_std,rtp_p90,1vd_mean,1vd_std,...,ss_ag_ppm_mean,ss_sb_ppm_mean,ss_fe_pct_mean,ss_mn_ppm_mean,ss_cr_ppm_mean,ss_has_any_mean,ss_stream_count,has_rock,has_soil,has_stream
0,0.0,0,-341.819104,1.742753,-339.439493,-34.246340,1.099034,-32.876038,-0.011716,0.000399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0.0,1,-343.149000,1.733180,-340.917496,-32.579243,1.059147,-31.281596,-0.013229,0.000884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,0.0,2,-344.533743,1.655959,-342.339349,-29.768549,1.097842,-28.366481,-0.014983,0.000292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,0.0,3,-345.051959,1.430893,-343.094437,-26.194259,1.119967,-24.623184,-0.014757,0.000290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,0.0,4,-345.027289,1.206863,-343.312057,-21.781616,1.419676,-19.703737,-0.014142,0.000449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [84]:

float64_nullable_cols = df.select_dtypes(include=["Float64"]).columns
print("Nullable Float64 cols:", list(float64_nullable_cols)[:20], " ... total:", len(float64_nullable_cols))


df[float64_nullable_cols] = df[float64_nullable_cols].astype("float64")


print(df[float64_nullable_cols].dtypes.head())

Nullable Float64 cols: ['label', 'tmi_mean', 'tmi_std', 'tmi_p90', 'rtp_mean', 'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std', '1vd_p90', 'dist_mag_line_m', 'mag_nearest_amp', 'mag_len_5km', 'mag_amp_max_5km', 'dist_grav_line_m', 'grav_nearest_amp', 'grav_len_5km', 'grav_amp_max_5km', 'dem_mean', 'dem_std']  ... total: 99


ValueError: Columns must be same length as key

In [85]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5358536 entries, 0 to 5358535
Columns: 105 entries, label to has_stream
dtypes: Float64(12), float64(87), geometry(1), int32(1), int64(4)
memory usage: 4.2 GB
